### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!pip install langchain

In [4]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [6]:
!pip install langchain_groq

In [7]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002732721D040>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002732721DA30>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
!pip install langchain_core

In [9]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [10]:
result

AIMessage(content="The literal translation would be:\n\n* **Bonjour** (Hello)\n* **Comment allez-vous?** (How are you?)\n\nHowever, depending on the context and who you are speaking to, you might use more informal options:\n\n* **Salut !** (Hi!)\n* **Ça va ?** (How's it going?) \n\n\nRemember to choose the greeting that is most appropriate for the situation.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 21, 'total_tokens': 111, 'completion_time': 0.163636364, 'prompt_time': 0.000149639, 'queue_time': 0.021403931, 'total_time': 0.163786003}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-688079cd-bd0d-4745-abd4-dd1de15b47ac-0', usage_metadata={'input_tokens': 21, 'output_tokens': 90, 'total_tokens': 111})

In [11]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

"The literal translation would be:\n\n* **Bonjour** (Hello)\n* **Comment allez-vous?** (How are you?)\n\nHowever, depending on the context and who you are speaking to, you might use more informal options:\n\n* **Salut !** (Hi!)\n* **Ça va ?** (How's it going?) \n\n\nRemember to choose the greeting that is most appropriate for the situation.\n"

In [12]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Salut, comment vas-tu ?** (Informal, singular)\n* **Bonjour, comment allez-vous ?** (Formal, singular)\n* **Bonjour, comment allez-vous ?** (Formal, plural)\n\n\nLet me know if you\'d like to see more variations!\n'

In [13]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [14]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [15]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [16]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [25]:
!pip install streamlit